# **E-Commerce Growth Insights with SQL Window Functions**

**Scenario**

You are a Data Analyst at an e-commerce company.
The company tracks customer purchases and the employees handling those sales.
Management wants to uncover patterns such as first-time buyers, top customers, top-performing employees, and sales trends.

Your challenge is to answer these business-driven questions using SQL window functions.

**Dataset**

A fictional Orders dataset was created for this project, containing:

OrderID – unique identifier for each transaction

CustomerID – customer who placed the order

EmployeeName – sales rep handling the order

Region – customer region

OrderDate – transaction date

Amount – purchase amount

**Business Questions**

*Customer Insights*

What was each customer’s first purchase?

What was their most recent purchase?

How much has each customer spent in running total over time?

*Employee Insights*

What is the running total of sales per employee?

Which employees are the top performers ranked by sales?

What are the top 2 sales deals handled by each employee?

*Trend Insights*

How does each employee’s sales change month over month?

Which customers spent the most each month, and how do they rank against others?

What were the best and worst sales recorded for each employee?

**Key Learnings**

- Practiced SQL window functions in a realistic business scenario.  
- Adapted queries for SQLite (`strftime`, CTEs).  
- Extracted customer, employee, and trend insights.  


%config SqlMagic.style = '_DEPRECATED_DEFAULT' is required in this, otherwise table wont show up in SELECT statements

In [1]:
!pip install ipython-sql
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT' 
%sql sqlite://

In [2]:
%%sql

CREATE TABLE Orders (
    OrderID INT,
    CustomerID Varchar(10),
    EmployeeName Varchar (100),
    Region Varchar(50),
    OrderDate Date,
    Amount INT)

 * sqlite://
Done.


[]

In [3]:
%%sql

INSERT INTO Orders VALUES (1,'C1','Alice','North','2023-01-10',250);
INSERT INTO Orders VALUES (2,'C2','Bob','South','2023-01-12',100);
INSERT INTO Orders VALUES (3,'C1','Alice','North','2023-02-01',300);
INSERT INTO Orders VALUES (4,'C3','Charlie','East','2023-02-05',150);
INSERT INTO Orders VALUES (5,'C2','Bob','South','2023-02-07',200);
INSERT INTO Orders VALUES (6,'C1','Alice','North','2023-03-15',400);
INSERT INTO Orders VALUES (7,'C3','Charlie','East','2023-03-20',500);
INSERT INTO Orders VALUES (8,'C2','Bob','South','2023-03-25',350);
INSERT INTO Orders VALUES (9,'C4','Charlie','East','2023-03-28',600);
INSERT INTO Orders VALUES (10,'C1','Alice','North','2023-04-05',450);
INSERT INTO Orders VALUES (11,'C2','Bob','South','2023-04-06',700);
INSERT INTO Orders VALUES (12,'C5','Alice','North','2023-04-10',800);

 * sqlite://
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [4]:
%%sql

SELECT *
FROM Orders;

 * sqlite://
Done.


OrderID,CustomerID,EmployeeName,Region,OrderDate,Amount
1,C1,Alice,North,2023-01-10,250
2,C2,Bob,South,2023-01-12,100
3,C1,Alice,North,2023-02-01,300
4,C3,Charlie,East,2023-02-05,150
5,C2,Bob,South,2023-02-07,200
6,C1,Alice,North,2023-03-15,400
7,C3,Charlie,East,2023-03-20,500
8,C2,Bob,South,2023-03-25,350
9,C4,Charlie,East,2023-03-28,600
10,C1,Alice,North,2023-04-05,450


# **Customer Insights**

**First Purchase per Customer**

In [5]:
%%sql

SELECT CustomerID, OrderDate, Amount, First_Value(Amount) OVER (PARTITION BY CustomerID ORDER BY OrderDate) As FirstPurchase
From Orders;

 * sqlite://
Done.


CustomerID,OrderDate,Amount,FirstPurchase
C1,2023-01-10,250,250
C1,2023-02-01,300,250
C1,2023-03-15,400,250
C1,2023-04-05,450,250
C2,2023-01-12,100,100
C2,2023-02-07,200,100
C2,2023-03-25,350,100
C2,2023-04-06,700,100
C3,2023-02-05,150,150
C3,2023-03-20,500,150


**Most Recent Purchase per Customer**

In [7]:
%%sql
SELECT CustomerID, OrderDate, Amount, Last_Value(Amount) OVER (PARTITION BY CustomerID ORDER BY OrderDate ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) As LatestPurchase
From Orders;

 * sqlite://
Done.


CustomerID,OrderDate,Amount,LatestPurchase
C1,2023-01-10,250,450
C1,2023-02-01,300,450
C1,2023-03-15,400,450
C1,2023-04-05,450,450
C2,2023-01-12,100,700
C2,2023-02-07,200,700
C2,2023-03-25,350,700
C2,2023-04-06,700,700
C3,2023-02-05,150,500
C3,2023-03-20,500,500


**Running Total per Customer**

In [8]:
%%sql
SELECT CustomerID, OrderDate, Amount, Sum(Amount) OVER (PARTITION BY CustomerID ORDER BY OrderDate) AS RunningTotal
FROM Orders;

 * sqlite://
Done.


CustomerID,OrderDate,Amount,RunningTotal
C1,2023-01-10,250,250
C1,2023-02-01,300,550
C1,2023-03-15,400,950
C1,2023-04-05,450,1400
C2,2023-01-12,100,100
C2,2023-02-07,200,300
C2,2023-03-25,350,650
C2,2023-04-06,700,1350
C3,2023-02-05,150,150
C3,2023-03-20,500,650


# **Employee Insights**

**Running Total per Employee**

In [14]:
%%sql

SELECT EmployeeName, OrderDate, Amount, SUM(Amount) OVER (PARTITION BY EmployeeName ORDER BY OrderDate) AS EmployeeRunningTotal
FROM Orders;

 * sqlite://
Done.


EmployeeName,OrderDate,Amount,EmployeeRunningTotal
Alice,2023-01-10,250,250
Alice,2023-02-01,300,550
Alice,2023-03-15,400,950
Alice,2023-04-05,450,1400
Alice,2023-04-10,800,2200
Bob,2023-01-12,100,100
Bob,2023-02-07,200,300
Bob,2023-03-25,350,650
Bob,2023-04-06,700,1350
Charlie,2023-02-05,150,150


**Rank Employees by Total Sales**

In [21]:
%%sql

SELECT EmployeeName, OrderDate, SUM(Amount) AS TotalSales, RANK() OVER(ORDER BY SUM(Amount) DESC) As SalesRank
FROM Orders
GROUP BY EmployeeName;

 * sqlite://
Done.


EmployeeName,OrderDate,TotalSales,SalesRank
Alice,2023-01-10,2200,1
Bob,2023-01-12,1350,2
Charlie,2023-02-05,1250,3


**Top 2 Sales per Employee**

In [35]:
%%sql

WITH TopSales AS
(SELECT EmployeeName, OrderDate, Amount, ROW_NUMBER() OVER(PARTITION BY EmployeeName ORDER BY Amount DESC) AS Top2
FROM Orders)
SELECT *
FROM TopSales
where Top2<=2;

 * sqlite://
Done.


EmployeeName,OrderDate,Amount,Top2
Alice,2023-04-10,800,1
Alice,2023-04-05,450,2
Bob,2023-04-06,700,1
Bob,2023-03-25,350,2
Charlie,2023-03-28,600,1
Charlie,2023-03-20,500,2


# **Trend Insights**

**Month-over-Month Change per Employee**

In [39]:
%%sql

SELECT EmployeeName, 
OrderDate, 
Amount, 
LAG(Amount) OVER(PARTITION BY EmployeeName ORDER BY OrderDate) As PrevSale,
Amount-LAG(Amount) OVER (PARTITION BY EmployeeName ORDER BY OrderDate) As ChangeinSale
FROM Orders;

 * sqlite://
Done.


EmployeeName,OrderDate,Amount,PrevSale,ChangeinSale
Alice,2023-01-10,250,None,None
Alice,2023-02-01,300,250,50
Alice,2023-03-15,400,300,100
Alice,2023-04-05,450,400,50
Alice,2023-04-10,800,450,350
Bob,2023-01-12,100,None,None
Bob,2023-02-07,200,100,100
Bob,2023-03-25,350,200,150
Bob,2023-04-06,700,350,350
Charlie,2023-02-05,150,None,None


**Monthly Spending by Customer with Ranking**

Month(OrderDate) AS OrderMonth works in SSMS but not in SQLLite 

In [55]:
%%sql

SELECT CustomerID, 
strftime('%m', OrderDate),
Sum(Amount) AS MonthlySpend, 
RANK() OVER(PARTITION BY strftime('%m', OrderDate) ORDER BY Sum(Amount) DESC) AS MonthlyRank
FROM Orders
GROUP BY CustomerID, strftime('%m', OrderDate);

 * sqlite://
Done.


CustomerID,"strftime('%m', OrderDate)",MonthlySpend,MonthlyRank
C1,01,250,1
C2,01,100,2
C1,02,300,1
C2,02,200,2
C3,02,150,3
C4,03,600,1
C3,03,500,2
C1,03,400,3
C2,03,350,4
C5,04,800,1


**Best & Worst Sale per Employee**

In [63]:
%%sql

WITH SalesWithExtremes AS (
SELECT EmployeeName, 
OrderDate, 
Amount, 
FIRST_VALUE(Amount) OVER (PARTITION BY EmployeeName ORDER BY Amount ASC) AS WorstSale,
FIRST_VALUE(Amount) OVER (PARTITION BY EmployeeName ORDER BY Amount DESC) As BestSale
From Orders)
SELECT DISTINCT EmployeeName, WorstSale, BestSale
FROM SalesWithExtremes;

 * sqlite://
Done.


EmployeeName,WorstSale,BestSale
Alice,250,800
Bob,100,700
Charlie,150,600
